# This is all over the place, don't worry will be fixed

In [127]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from matplotlib import pyplot
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
import string
import re
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,\
HashingVectorizer
%matplotlib inline

In [128]:
# Functions that will be used within the notebook
def onehotencoder(column_name, df):
    column_name_ohe = df[[column_name]]
    ohe = OneHotEncoder(categories = 'auto', sparse = False)
    ohe.fit(column_name_ohe)
    column_transformed = ohe.transform(column_name_ohe)
    column_encoded_ohe = pd.DataFrame(column_transformed, columns = ohe.get_feature_names([column_name]), index = df.index)
    return column_encoded_ohe

def list_contains(List1, List2): 
    check = False
  
    # Iterate in the 1st list 
    for m in List1: 
  
        # Iterate in the 2nd list 
        for n in List2: 
    
            # if there is a match
            if m == n: 
                check = True
                return check  
                  
    return check

Loading in the data

In [129]:
df= pd.read_csv('CraigslistCars.csv')

MemoryError: 

In [ ]:
df.isna().sum().sum()

In [ ]:
df_1 = df.copy()

In [ ]:
df_1.head()
# visually seeing the first 5 rows of the raw dataset

From the first few lines we are seeing that there are many rows that have NaN as a value within a column.

In [ ]:
# Will take a random sample of a row to see if an arbritrary row will also follow suit of the first 5 rows by having 
# multiple NaN values
df_1.sample()

As you may possibly see above at least one column could have a NaN value in a row.

In [ ]:
# Want to see what are the 25 columns that will be dealt with. Also will see which should be the target for this dataset
df_1.columns

Seeing that within the dataset that price exists and therefore should be a solid target to predict.

In [ ]:
# Seeing the range of the prices
df_1.price.min(), df_1.price.max()

In [ ]:
# The range of price within the dataset goes from 0 to 3,736,928,711
# Want to check out the years of these vehicles because some older cars have been valued in the millions
df_1.loc[df_1['price'] == df_1.price.min(), ['year']], df_1.loc[df_1['price']==df_1.price.max(), ['year']]

In [ ]:
b =sns.boxplot( df_1['price'] )
b.set(xlabel = "Price (in $)", title='Price Distribution')
sns.set_style("dark");

Seeing that over 30,000 rows have the price of 0 and 2 have the large value it is showing the dataset is far fromm perfect

Best way to have the price of 0 not sway modeling later on is to limit price values as well as these strange values that are in the billions. Therefore will take an inner 90% of the data.

In [ ]:
df_1['price'] = df_1['price'][df_1['price'].between(df_1['price'].quantile(.085), df_1['price'].quantile(.985),inclusive=True)]
df_1.price.min(), df_1.price.max()
# shows 100 - 59999 is an inner 90%%

In [ ]:
# Want new df with just price values that range from 100 to 59999

In [ ]:
# Making new df of just values of cars between 100 and 59,999
df_2 = df_1.loc[(df_1['price']>100) & (df_1['price'] <59999)].copy()
df_2.head(), len(df_2)

# Dealing with all the possible NaN values in the dataset

In [ ]:
df_2.isna().sum()

year, manufacturer, model, condition, cylinders, fuel, odometer, title_status, transmission, VIN, drive, size, type, paint_color
image_url, description, county, state, lat, long, posting_date have Na values and some have around 40% of its rows with NaN values.

In [ ]:
# Checking the 'county' column since it has the most NaN values.
df_2['county'].value_counts()
# is completely empty

In [ ]:
# The column is not going to be useful therefore it is safe to just drop it
df_2.drop('county', axis=1, inplace=True)
df_2.head()

County dealt with

The 'year' column is the first column that shows it has NaN values so lets take a deeper dive into the column

In [ ]:
df_2['year'].value_counts()

In [ ]:
# Checking the oldest car
df_2['year'].min()

In [ ]:
# Checking newest car
df_2.year.max()

In [ ]:
# Making a sorted list of all the years of the cars represented in the dataset
unique_years = df_2['year'].unique()
print(sorted(unique_years))

Barring a couple of years in the early 1900's, the dataset seems to have nearly every year in the 20th century and all years that have occured in the 21st century.

In [ ]:
df_2.year.value_counts(normalize=True)
# most of the cars stated year is within the past 6 years

In [ ]:
p=sns.boxplot(df_2.year)
p.set(xlabel = 'Car Year ', title='Year Distribution')
sns.set_style("dark");

Want to remove some of the lower outliers that exist in the year column so will take just price an inner 90% of the data.

In [ ]:
df_top_years = df_2.copy()
df_top_years['year'] = df_top_years['year'][df_top_years['year'].between(df_top_years['year'].quantile(.1), df_top_years['year'].quantile(1),inclusive=True)]
df_top_years.year.min(), df_top_years.year.max()

We come to see that the lower end of starting years for cars that we should be looking at is after the year 2002. Anything prior could mess up the data.

## Making new Df

In [ ]:
# grabbing only values with years greater than 2002
df_limited_years= df_2.loc[df_2['year'] >2002].copy()

In [ ]:
# df_u = df_limited_years.copy()

In [ ]:
# standard = StandardScaler()

In [ ]:
# scaleddddd = standard.fit_transform(df_u[['year']].values)
# df_u['year']= scaleddddd

In [ ]:
# x = standard.inverse_transform(df_u['year'])

In [ ]:
# x

In [ ]:
# scaleddddd.mean(), scaleddddd.std()

In [ ]:
# cc = (2022-x.mean())/ x.std()
# cc

In [ ]:
# df_limited_years.year.value_counts()

In [ ]:
df_limited_years.isna().sum()

Now that two key columns have been dealt with it is time to start looking at the rest of the dataset and to find ways for the empty values to be filled in.

In [ ]:
# A new df is being made with all the rows where the condition column does not have an NaN value.
df_condition = df_limited_years.loc[df_limited_years['condition'].notna()].copy()
# Taking the df above and focusing on where the 'model' column also doesn't have any NaN values.
df_model_and_condition = df_condition.loc[df_condition['model'].notna()].copy()
# Making an empty dictionary
condition_dict = {}
# This for loop will grab all the associated values of model and condition of the vehicle and save it within the dictionary 
# above
for a, b in df_model_and_condition[['model', 'condition']].values:
    condition_dict[a] = b
# Here with the dictionary above it will run through the 'condition' column and fill out missing values with appropiate ones
# found within the made dictionary above
df_limited_years['condition'] = df_limited_years.apply(lambda x: x['condition'] if pd.isna(x['model']) or pd.notna(x['condition']) else condition_dict[x['model']] if x['model'] in condition_dict.keys() else x['condition'], axis=1)

In [ ]:
# We see that it filled in 116,072 NaN values within the 'condition' column
df_limited_years.isna().sum()

In [ ]:
# Will do same procedure as abpve with the 'cylinders' column
df_cylinders = df_limited_years.loc[df_limited_years['cylinders'].notna()].copy()
df_cylinders_model = df_cylinders.loc[df_cylinders['model'].notna()].copy()
cylinders_dictionary = {}
for c, d in df_cylinders_model[['model', 'cylinders']].values:
    cylinders_dictionary[c] = d
df_limited_years['cylinders'] = df_limited_years.apply( lambda x: x['cylinders'] if pd.isna( x['model'] ) or pd.notna( x['cylinders'] ) else cylinders_dictionary[x['model']] if x['model'] in cylinders_dictionary.keys() else x['cylinders'], axis=1)

In [ ]:
df_limited_years.isna().sum()
# 109,649 NaN values filled in

In [ ]:
# Same procedure as down above
df_manufacturer = df_limited_years.loc[df_limited_years['manufacturer'].notna()].copy()
df_manufacturer_model = df_manufacturer.loc[df_manufacturer['model'].notna()].copy()
manufacturer_dictionary = {}
for e, f in df_manufacturer_model[['model', 'manufacturer']].values:
    manufacturer_dictionary[e] = f
df_limited_years['manufacturer'] = df_limited_years.apply( lambda x: x['manufacturer'] if pd.isna( x['model'] ) or pd.notna( x['manufacturer'] ) else manufacturer_dictionary[x['model']] if x['model'] in manufacturer_dictionary.keys() else x['manufacturer'], axis=1)

In [ ]:
df_limited_years.isna().sum()
# 445 NaN values filled in

In [ ]:
# A groupy forward fill can be applied here to use the model column to appropiately fill in other columns with missing values
df_limited_years['cylinders'] = df_limited_years.groupby('model')['cylinders'].ffill()
df_limited_years['manufacturer'] = df_limited_years.groupby('model')['manufacturer'].ffill()
df_limited_years['fuel']=df_limited_years.groupby('model')['fuel'].ffill()
df_limited_years['odometer'] = df_limited_years['odometer'].fillna(df_limited_years.groupby(['year','model'])['odometer'].transform('mean'))
df_limited_years['size']= df_limited_years.groupby('model')['size'].ffill()
df_limited_years['type']= df_limited_years.groupby('model')['type'].ffill()
df_limited_years['manufacturer']= df_limited_years.groupby('model')['manufacturer'].ffill()

In [ ]:
df_limited_years.isna().sum()
# cylinders filled in 1,410 NaN values
# condition filled in none
# fuel filled in 816 NaN values
# odometer with transforming and using the mean filled in 1658
# size filled in 162,271 NaN values
# type filled in 59,738 NaN values
# manufacturer filled in none 

Manufacturer is a very important column so want to see what car makers are in the column.

In [ ]:
unique_manufacturer = df_2['manufacturer'].unique()
manufacturer_list = list(unique_manufacturer)
manufacturer_list =manufacturer_list[1:]
manufacturer_list

Seeing there are a variety of car makers from around the world exists in the dataset then it is key to find ways to fill in the column.

Through domain knowledge the description of the vehicle is bound to have the name of the car maker within it. Below I will check to see if that is the case.

In [ ]:
# Taking in a sample of a description of a row whose manufacturer value is NaN to see if the value is is in the description 
# column.
sample = df_2['description'][97]

In [ ]:
len(sample)
# Checking the length. 

In [ ]:
# Seeing that it is 4573 in length and it contains punction will use regexptokenizer to remove punctions and simply the 
# sample
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tokenizer = RegexpTokenizer(pattern)
sample_doc = tokenizer.tokenize(sample)
sample_doc = [token.lower() for token in sample_doc]
sw =stopwords.words('english')
sample_doc = [token for token in sample_doc if token not in sw]

In [ ]:
len(set(sample_doc))
# The length of the sample has decresed to 252 in length making it easier on the machine to run through and search.

In [ ]:
type(sample_doc)

In [ ]:
# Using the function list_contains to see if it true whether the sample doc has at least one of the strings in the
# manufacturer list
List1 = unique_manufacturer
List2 = sample_doc 
print("Test Case#1 ", list_contains(List1, List2)) 

Comes out to be that the description does have a car makers name within it.

In [ ]:
# Seeing which car maker is within the desciption
car = set(unique_manufacturer) & set(sample_doc)
car

With it proven that the car makers name can be found with the description of the vehicle will now use the description column to help fill out the NaN values within the manufacturer column.

In [ ]:
# Making a new df where it is grabbing all the rows where the manufacturer is NaN and grabbing its associated description
# and id column
manu_desc=df_limited_years.loc[df_limited_years['manufacturer'].isna(), ['description','manufacturer', 'id']].copy()
manu_desc.head()

In [ ]:
# Lowercasing and making strings to all the values within the description column since the list of manufacturer are all
# lowercase
manu_desc['description']=manu_desc['description'].str.lower().str.split()

In [ ]:
# checking to see if it worked
manu_desc.head()

In [ ]:
# Now making for loops to run through and fill in an empty list with the common values
index_nums=[]
m =[]
for index, row in manu_desc.iterrows():   #runs through all the rows grabbing the index and the complete value
    for manufacturer in manufacturer_list: #runs through the manufacturer list 
        if manufacturer in row['description']:  #seeing if the value has a common value from the list
            index_nums.append(index)        #appending its index to index_nums empty list from above
            m.append(manufacturer)          #appending the assocaited values from the list and row 

In [ ]:
# Here was just showing the result of the loop showing that it work. But it is very long and takes up a lot of space on the
# repo so it is commented out. For those cloning it down you can uncomment it and look
# result = [None]*(len(index_nums)+len(m))
# result[::2] = index_nums
# result[1::2] =m

# for x, y in zip(result[::2], result[1::2]):
    
#     print(x,y)

In [ ]:
# Making a new df from the values grabbed from the for loop above to now associate the row with the new value
manu_desc_completed = pd.DataFrame(index_nums, columns=['index'])    
manu_desc_completed['manufacturer'] = m

In [ ]:
manu_desc_completed.head(15)

In [ ]:
manu_desc_completed_new=manu_desc_completed.drop_duplicates(subset=['index']) #just grabbing first appearance of whatever
# value came from the for loop since cars can have more than one maker
manu_desc_completed_new.manufacturer.value_counts()

In [ ]:
# manu_desc_completed_new.set_index('index',inplace=True ) reseting the index to the index column made above
manu_desc_completed_new.head()

In [ ]:
df_yo= df_limited_years.copy() #making a new df

In [ ]:
df_yo.update(manu_desc_completed_new)  #Using the the update method to fill in the missing values that were generated from
# operations above
df_yo.isna().sum()
# We see 2,716 NaN values were filled in.

Seeing that the description column could fill in NaN values for the manufacturer it probably can work for other columns as well. The same steps from the procedure above will be used on other columns that are deemed fit to have this work.

In [ ]:
# Making a list of the condition of the vehicle
unique_condition = df_2['condition'].unique()
condition_list = list(unique_condition)
condition_list =condition_list[1:]
condition_list

In [ ]:
# New df with all rows of NaN values from condition
cond_desc=df_limited_years.loc[df_limited_years['condition'].isna(), ['description','condition', 'id']].copy()
cond_desc.head()

In [ ]:
cond_desc['description']=cond_desc['description'].str.lower().str.split()
# lowering the description column

In [ ]:
# Grabbing index and value of occurance
index_nums2=[]
c =[]
for index, row in cond_desc.iterrows():
    for condition in condition_list:
        if condition in row['description']:
            index_nums2.append(index)
            c.append(condition)  

In [ ]:
# Same as above with printed out version of it working
# result2 = [None]*(len(index_nums2)+len(c))
# result2[::2] = index_nums2
# result2[1::2] =c

# for x, y in zip(result2[::2], result2[1::2]):
    
#     print(x,y)

In [ ]:
cond_desc_completed = pd.DataFrame(index_nums2, columns=['index'])
cond_desc_completed['condition'] = c
cond_desc_completed_new=cond_desc_completed.drop_duplicates(subset=['index'])
cond_desc_completed_new.set_index('index',inplace=True )
cond_desc_completed_new.head()

In [ ]:
df_yo.update(cond_desc_completed_new)
df_yo.isna().sum()

In [ ]:
# unique_cylinders = df_2['cylinders'].unique()
# cylinders_list = list(unique_cylinders)
# cylinders_list = cylinders_list[1:]
# cylinders_list

In [ ]:
# cyld_desc=df_limited_years.loc[df_limited_years['cylinders'].isna(), ['description','cylinders', 'id']].copy()
# cyld_desc.head()

In [ ]:
# cyld_desc['description']=cyld_desc['description'].str.lower().str.split()

In [ ]:

# ll=[]
# cy =[]
# for index, row in cyld_desc.iterrows():
#     for cylinders in cylinders_list:
#         if cylinders in row['description']:
#             ll.append(index)
#             cy.append(manufacturer)  
# result3 = [None]*(len(ll)+len(cy))
# result3[::2] = ll
# result3[1::2] =cy
# result3
# # for x, y in zip(result3[::2], result3[1::2]):
    
# #     print(x,y)

In [ ]:
unique_fuel = df_limited_years['fuel'].unique()
fuel_list = list(unique_fuel)
fuel_list = ['gas', 'diesel', 'hybrid', 'electric']

In [ ]:
fuel_desc=df_limited_years.loc[df_limited_years['fuel'].isna(), ['description','fuel', 'id']].copy()
fuel_desc['description']=fuel_desc['description'].str.lower().str.split()
fuel_desc.head()

In [ ]:
index_nums4=[]
f =[]
for index, row in fuel_desc.iterrows():
    for fuel in fuel_list:
        if fuel in row['description']:
            index_nums4.append(index)
            f.append(fuel)  

In [ ]:
# result4 = [None]*(len(index_nums4)+len(f))
# result4[::2] = index_nums4
# result4[1::2] =f

# for x, y in zip(result4[::2], result4[1::2]):
    
#     print(x,y)

In [ ]:
fuel_desc_completed = pd.DataFrame(index_nums4, columns=['index'])
fuel_desc_completed['fuel'] = f
fuel_desc_completed_new=fuel_desc_completed.drop_duplicates(subset=['index'])
fuel_desc_completed_new.set_index('index',inplace=True )
fuel_desc_completed_new.head()

In [ ]:
df_yo.update(fuel_desc_completed_new)
df_yo.isna().sum()

In [ ]:
unique_transmission = df_limited_years['transmission'].unique()
trasnmission_list = list(unique_transmission)
transmission_list = ['automatic', 'manual']

In [ ]:
tran_desc=df_limited_years.loc[df_limited_years['transmission'].isna(), ['description','transmission', 'id']].copy()
tran_desc['description']=tran_desc['description'].str.lower().str.split()
tran_desc.head()

In [ ]:
index_nums5=[]
t =[]
for index, row in tran_desc.iterrows():
    for transmission in transmission_list:
        if transmission in row['description']:
            index_nums5.append(index)
            t.append(transmission)  

In [ ]:
# result5 = [None]*(len(index_nums5)+len(t))
# result5[::2] = index_nums5
# result5[1::2] =t

# for x, y in zip(result5[::2], result5[1::2]):
    
#     print(x,y)

In [ ]:
tran_desc_completed = pd.DataFrame(index_nums5, columns=['index'])
tran_desc_completed['transmission'] = t
tran_desc_completed_new=tran_desc_completed.drop_duplicates(subset=['index'])
tran_desc_completed_new.set_index('index',inplace=True )
tran_desc_completed_new.head()

In [ ]:
df_yo.update(tran_desc_completed_new)
df_yo.isna().sum()

In [ ]:
unique_drive = df_limited_years['drive'].unique()
drive_list = list(unique_drive)
drive_list= drive_list[1:]
drive_list

In [ ]:
driv_desc=df_limited_years.loc[df_limited_years['drive'].isna(), ['description','drive', 'id']].copy()
driv_desc['description']=driv_desc['description'].str.lower().str.split()
driv_desc.head()

In [ ]:
driv_desc.isna().sum()
# Seeing if any NaN values exist within the description column because it will interfere with the for loop
# There is and that row will be dropped

In [ ]:
driv_desc.dropna(subset=['description'], axis=0, inplace=True)
# dropping the row

In [ ]:
index_nums6=[]
d =[]
for index, row in driv_desc.iterrows():
    for drive in drive_list:
        if drive in row['description']:
            index_nums6.append(index)
            d.append(drive)  

In [ ]:
# result6 = [None]*(len(index_nums6)+len(d))
# result6[::2] = index_nums6
# result6[1::2] =d

# for x, y in zip(result6[::2], result6[1::2]):
    
#     print(x,y)

In [ ]:
driv_desc_completed = pd.DataFrame(index_nums6, columns=['index'])
driv_desc_completed['drive'] = d
driv_desc_completed_new=driv_desc_completed.drop_duplicates(subset=['index'])
driv_desc_completed_new.set_index('index',inplace=True )
driv_desc_completed_new.head()

In [ ]:
df_yo.update(driv_desc_completed_new)
df_yo.isna().sum()

In [ ]:
unique_size = df_limited_years['size'].unique()
size_list = list(unique_size)
size_list= size_list[1:]
size_list

In [ ]:
size_desc=df_limited_years.loc[df_limited_years['size'].isna(), ['description','size', 'id']].copy()
size_desc['description']=size_desc['description'].str.lower().str.split()
size_desc.head()

In [ ]:
index_nums7=[]
s =[]
for index, row in size_desc.iterrows():
    for size in size_list:
        if size in row['description']:
            index_nums7.append(index)
            s.append(size)  

In [ ]:
# result7 = [None]*(len(index_nums7)+len(s))
# result7[::2] = index_nums7
# result7[1::2] =s

# for x, y in zip(result7[::2], result7[1::2]):
    
#     print(x,y)

In [ ]:
size_desc_completed = pd.DataFrame(index_nums7, columns=['index'])
size_desc_completed['size'] = s
size_desc_completed_new=size_desc_completed.drop_duplicates(subset=['index'])
size_desc_completed_new.set_index('index',inplace=True )
size_desc_completed_new.head()

In [ ]:
df_yo.update(size_desc_completed_new)
df_yo.isna().sum()

In [ ]:
df_yo['type'].value_counts()

In [ ]:
unique_type = df_limited_years['type'].unique()
type_list = list(unique_type)
# type_list= type_list[1:]
type_list = ['pickup',
 'truck',
 'other',
 'coupe',
 'SUV',
 'hatchback',
 'mini-van',
 'sedan',
 'offroad',
 'bus',
 'convertible',
 'wagon',
 'van']

In [ ]:
type_desc=df_limited_years.loc[df_limited_years['type'].isna(), ['description','type', 'id']].copy()
type_desc['description']=type_desc['description'].str.lower().str.split()
type_desc.head()

In [ ]:
index_nums8=[]
ty =[]
for index, row in type_desc.iterrows():
    for types in type_list:
        if types in row['description']:
            index_nums8.append(index)
            ty.append(types)  

In [ ]:
# result8 = [None]*(len(index_nums8)+len(ty))
# result8[::2] = index_nums8
# result8[1::2] =ty

# for x, y in zip(result8[::2], result8[1::2]):
    
#     print(x,y)

In [ ]:
type_desc_completed = pd.DataFrame(index_nums8, columns=['index'])
type_desc_completed['type'] = ty
type_desc_completed_new=type_desc_completed.drop_duplicates(subset=['index'])
type_desc_completed_new.set_index('index',inplace=True )
type_desc_completed_new.head()

In [ ]:
df_yo.update(type_desc_completed_new)
df_yo.isna().sum()

In [ ]:
df_yo.drop(['id', 'url', 'region_url', 'VIN', 'image_url', 'description', 'lat', 'long', 'posting_date'], axis=1, inplace=True)
df_yo.head()

In [107]:
df_yo1 = df_yo.copy()
df_yo1.dropna(inplace=True)
df_yo1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151003 entries, 31 to 426872
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        151003 non-null  object 
 1   price         151003 non-null  float64
 2   year          151003 non-null  float64
 3   manufacturer  151003 non-null  object 
 4   model         151003 non-null  object 
 5   condition     151003 non-null  object 
 6   cylinders     151003 non-null  object 
 7   fuel          151003 non-null  object 
 8   odometer      151003 non-null  float64
 9   title_status  151003 non-null  object 
 10  transmission  151003 non-null  object 
 11  drive         151003 non-null  object 
 12  size          151003 non-null  object 
 13  type          151003 non-null  object 
 14  paint_color   151003 non-null  object 
 15  state         151003 non-null  object 
dtypes: float64(3), object(13)
memory usage: 19.6+ MB


In [108]:
# import pickle


In [109]:
df_yo2=df_yo.copy()

In [110]:
df_yo2.drop(['drive', 'size', 'paint_color'], axis=1,inplace=True)
df_yo2.dropna(inplace=True)
df_yo2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290827 entries, 27 to 426878
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        290827 non-null  object 
 1   price         290827 non-null  float64
 2   year          290827 non-null  float64
 3   manufacturer  290827 non-null  object 
 4   model         290827 non-null  object 
 5   condition     290827 non-null  object 
 6   cylinders     290827 non-null  object 
 7   fuel          290827 non-null  object 
 8   odometer      290827 non-null  float64
 9   title_status  290827 non-null  object 
 10  transmission  290827 non-null  object 
 11  type          290827 non-null  object 
 12  state         290827 non-null  object 
dtypes: float64(3), object(10)
memory usage: 31.1+ MB


In [111]:
df_yo2.head()

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,type,state
27,auburn,33590.0,2014.0,gmc,sierra 1500 crew cab slt,good,8 cylinders,gas,57923.0,clean,other,pickup,al
28,auburn,22590.0,2010.0,chevrolet,silverado 1500,good,8 cylinders,gas,71229.0,clean,other,pickup,al
29,auburn,39590.0,2020.0,gmc,silverado 1500 crew,good,8 cylinders,gas,19160.0,clean,other,pickup,al
30,auburn,30990.0,2017.0,toyota,tundra double cab sr,good,8 cylinders,gas,41124.0,clean,other,pickup,al
31,auburn,15000.0,2013.0,ford,f-150 xlt,excellent,6 cylinders,gas,128000.0,clean,automatic,truck,al


In [112]:
df_yo1['paint_color'].value_counts()

white     37467
black     30766
silver    23450
grey      16762
blue      15528
red       14472
custom     4234
brown      3371
green      2951
orange      834
yellow      806
purple      362
Name: paint_color, dtype: int64

In [113]:
df_yo1.head()

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
31,auburn,15000.0,2013.0,ford,f-150 xlt,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,full-size,truck,black,al
55,auburn,19900.0,2004.0,ford,f250 super duty,good,8 cylinders,diesel,88000.0,clean,automatic,4wd,full-size,pickup,blue,al
59,auburn,14000.0,2012.0,gmc,odyssey,excellent,6 cylinders,gas,95000.0,clean,automatic,fwd,full-size,mini-van,silver,al
73,auburn,15000.0,2017.0,dodge,charger rt 4dr sedan,excellent,8 cylinders,gas,90000.0,rebuilt,automatic,rwd,mid-size,sedan,grey,al
86,auburn,3000.0,2004.0,chrysler,town & country,good,6 cylinders,gas,176144.0,clean,automatic,fwd,mid-size,mini-van,silver,al


In [ ]:
df_yo1 = df_yo1.loc[df_yo1['odometer'] < 500000]
df_yo1.info()

In [157]:
# a = onehotencoder('region' , df_yo1)
b = onehotencoder('manufacturer' , df_yo1)
# c = onehotencoder('model' , df_3)    too messy of a clumn therefore will be left out
d = onehotencoder('condition' , df_yo1)
e = onehotencoder('cylinders' , df_yo1)
f = onehotencoder('fuel' , df_yo1)
g = onehotencoder('title_status' , df_yo1)
h = onehotencoder('transmission' , df_yo1)
i = onehotencoder('drive' , df_yo1)
j = onehotencoder('size' , df_yo1)
k = onehotencoder('type', df_yo1)
l = onehotencoder('paint_color' , df_yo1)
m = onehotencoder('state', df_yo1)

In [115]:
df_yo1.isna().sum()

region          0
price           0
year            0
manufacturer    0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
size            0
type            0
paint_color     0
state           0
dtype: int64

In [135]:
e3 =(2022-df_yo1['year'].mean())/df_yo1['year'].std()
e3,df_yo1['year'].mean(),df_yo1['year'].std()

(2.1400507021695945, 2012.417203631716, 4.477836136577988)

In [149]:
df_yo1['odometer'].mean(), df_yo1['odometer'].std(), df_yo1['odometer'].max()

(102836.53701866485, 57728.5872305997, 499232.0)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150864 entries, 31 to 426872
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        150864 non-null  object 
 1   price         150864 non-null  float64
 2   year          150864 non-null  float64
 3   manufacturer  150864 non-null  object 
 4   model         150864 non-null  object 
 5   condition     150864 non-null  object 
 6   cylinders     150864 non-null  object 
 7   fuel          150864 non-null  object 
 8   odometer      150864 non-null  float64
 9   title_status  150864 non-null  object 
 10  transmission  150864 non-null  object 
 11  drive         150864 non-null  object 
 12  size          150864 non-null  object 
 13  type          150864 non-null  object 
 14  paint_color   150864 non-null  object 
 15  state         150864 non-null  object 
dtypes: float64(3), object(13)
memory usage: 19.6+ MB


In [158]:
df_final1 = pd.concat([df_yo1,b,d,e,f,g,h,i,j,k,l,m], axis=1)
df_final1.drop(['drive','region', 'manufacturer', 'condition', 'cylinders', 'fuel', 'title_status', 'transmission', 'size', 'type', 'state', 'paint_color'], axis=1, inplace=True)

In [162]:
col_ss = ['year']
ss = StandardScaler()
Scaled = ss.fit_transform(df_final1[col_ss])
Scaled = pd.DataFrame(Scaled, index=df_final1.index)
Scaled.columns = col_ss
df_final1['year'] = Scaled['year']


col_sss = ['odometer']
Scaledd = ss.fit_transform(df_final1[col_sss])
Scaledd = pd.DataFrame(Scaledd, index=df_final1.index)
Scaledd.columns = col_sss
df_final1['odometer'] = Scaledd['odometer']

df_final1['price'] = np.log(df_final1['price'])


In [163]:
with open('cars.pkl', 'wb') as carpickle:
    pickle.dump(df_final1,carpickle)

In [159]:
df_final1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150864 entries, 31 to 426872
Columns: 156 entries, price to state_wy
dtypes: float64(155), object(1)
memory usage: 185.7+ MB


In [120]:
# a = onehotencoder('region' , df_yo2)
# b = onehotencoder('manufacturer' , df_yo2)
# # c = onehotencoder('model' , df_3) too messy of a column
# d = onehotencoder('condition' , df_yo2)
# e = onehotencoder('cylinders' , df_yo2)
# f = onehotencoder('fuel' , df_yo2)
# g = onehotencoder('title_status' , df_yo2)
# h = onehotencoder('transmission' , df_yo2)
# i = onehotencoder('type', df_yo2)
# j = onehotencoder('state', df_yo2)

In [121]:
# df_yo3 =df_yo2.copy()
# df_yo3.drop(['region', 'manufacturer', 'condition', 'cylinders', 'fuel', 'title_status', 'transmission','type', 'state'], axis=1, inplace=True)

In [122]:
# df_yo4 = pd.concat([a,b,d,e,f,g,h,i,j], axis=1)

In [123]:
# df_yo5 = pd.concat

In [124]:
# df_final2 = pd.concat([df_yo3,a,b,d,e,f,g,h,i,j], axis=1)
# df_final2.drop(['region', 'manufacturer', 'condition', 'cylinders', 'fuel', 'title_status', 'transmission','type', 'state'], axis=1, inplace=True)

In [125]:
# df_final2['price'] = np.log(df_final2['price'])

# scaledddddd = ss.fit_transform(df_final2[['odometer']].values)
# df_final2['odometer']= scaledddddd

# scaleddddd = ss.fit_transform(df_final2[['year']].values)
# df_final2['year']= scaleddddd

In [126]:
# with open('cars_witthout_drive_size_paint_NaN.pkl', 'wb') as carpickles:
#     pickle.dump(df_final2,carpickles)